# Milestone 4
### Annie Anand Jacob Stambaugh

## Part 1

### Location

In [1]:
%run location_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1936: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:956: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


### Day

In [7]:
%run day_dataflow.py

### Cases

In [8]:
%run cases_dataflow.py

## Part 2 Checking Primary + FK

location

In [1]:
%%bigquery
SELECT combined_key as pk_and_fk, COUNT(*) occurrences
FROM datamart.location_Dataflow
GROUP BY combined_key
HAVING COUNT(*) > 1;

,pk_and_fk,occurrences


day

In [2]:
%%bigquery
SELECT day_key as pk, COUNT(*) occurrences
FROM datamart.day_Dataflow
GROUP BY day_key
HAVING COUNT(*) > 1;

,pk,occurrences


cases

In [4]:
%%bigquery
SELECT day_key as pk, COUNT(*) occurrences
FROM datamart.cases_Dataflow
GROUP BY day_key
HAVING COUNT(*) > 1;

,pk,occurrences


acs - housing

In [5]:
%%bigquery
SELECT state as pk, COUNT(*) occurrences
FROM datamart.housing
GROUP BY state
HAVING COUNT(*) > 1;

,pk,occurrences


acs - economics

In [6]:
%%bigquery
SELECT state as pk, COUNT(*) occurrences
FROM datamart.economics
GROUP BY state
HAVING COUNT(*) > 1;

,pk,occurrences


acs - demographics

In [8]:
%%bigquery
SELECT state as pk, COUNT(*) occurrences
FROM datamart.demographics
GROUP BY state
HAVING COUNT(*) > 1;

,pk,occurrences


acs - social

In [10]:
%%bigquery
SELECT state as pk, COUNT(*) occurrences
FROM datamart.social
GROUP BY state
HAVING COUNT(*) > 1;

,pk,occurrences


## Part 3

### **Q1**
Looking at states and comparing Average cases+deaths per day to number of houses lacking plumbing (infrastructure) as we well how mnay people take public transportation to work (economics). We limit ten so notebook isnt long but when creating views we do not limit so we can compare entire U.S

In [35]:
%%bigquery
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day, house.Lacking_complete_plumbing_facilities,econ.Public_transportation
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state, house.Lacking_complete_plumbing_facilities,econ.Public_transportation
limit 10
;

,state,avg_deaths_per_day,avg_cases_per_day,Lacking_complete_plumbing_facilities,Public_transportation
0,Indiana,2.319485,55.569118,10233,29628
1,Minnesota,0.610169,14.992213,7180,100645
2,Oklahoma,0.931328,19.321805,6464,7609
3,Virginia,0.731587,26.850882,11024,186834
4,Vermont,1.427252,33.295612,1500,5324
5,Maryland,5.960383,211.010929,6054,246926
6,Tennessee,0.694259,39.449454,7098,19604
7,Louisiana,8.854914,200.845034,7479,23798
8,South Carolina,1.174800,47.588492,6473,13837
9,Alabama,0.922691,33.578313,6818,8867


### **Q2**
Looking at states and comparing Average cases+deaths per day to number of people working in service industry for that state. We limit ten so notebook isnt long but when creating views we do not limit so we can compare entire U.S

In [32]:
%%bigquery
SELECT L.state,max(C.confirmed) as greatest_cases_on_a_day,max(C.deaths) as greatest_deaths_on_a_day, (demo.White/demo.Total_Population) as pct_white, (demo.Black/demo.Total_Population) as pct_black, (demo.Asian/demo.Total_Population) as pct_asian, (demo.Hispanic/demo.Total_Population) as pct_hispanic,econ.Unemployment_Rate,(econ.No_health_insurance_coverage/demo.Total_Population) as pct_with_no_health_insurance 
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state
group by L.state,econ.Unemployment_Rate,pct_with_no_health_insurance,pct_white,pct_black,pct_asian,pct_hispanic
order by greatest_deaths_on_a_day desc
limit 10
;

,state,avg_deaths_per_day,avg_cases_per_day,Service_occupations
0,Iowa,0.318255,13.407926,255954
1,Ohio,2.102959,55.523866,965392
2,Utah,0.509985,72.149002,242066
3,Idaho,0.477982,25.467890,145329
4,Maine,0.790456,29.508299,119100
5,Texas,0.951145,44.547583,2391815
6,Alaska,0.302970,10.899010,58449
7,Hawaii,0.757225,58.335260,156717
8,Kansas,0.482822,12.089785,237238
9,Nevada,5.086514,147.139949,380595


### Q3

Looking at states and comparing Average cases+deaths per day to percentage of white people in that state as well as number of people workinf fin agriculuture. We limit ten so notebook isnt long but when creating views we do not limit so we can compare entire U.S

In [36]:
%%bigquery
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day, econ.Agriculture_forestry,(demo.White/demo.Total_Population) as pct_white
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state, econ.Agriculture_forestry,pct_white
limit 10;

,state,avg_deaths_per_day,avg_cases_per_day,Agriculture_forestry,pct_white
0,Indiana,2.319485,55.569118,41079,0.827647
1,Minnesota,0.610169,14.992213,61997,0.820548
2,Oklahoma,0.931328,19.321805,82289,0.724237
3,Virginia,0.731587,26.850882,34735,0.669514
4,Vermont,1.427252,33.295612,8209,0.937704
5,Maryland,5.960383,211.010929,17758,0.545333
6,Tennessee,0.694259,39.449454,28342,0.772115
7,Louisiana,8.854914,200.845034,77379,0.618394
8,South Carolina,1.174800,47.588492,21668,0.667053
9,Alabama,0.922691,33.578313,26729,0.678411


## Part 4 - Creating Views
We did not limit so we can entire trends for U.S!

### View 1

In [28]:
%%bigquery
CREATE VIEW reports.Infrastructure AS
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day, house.Lacking_complete_plumbing_facilities,econ.Public_transportation
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state, house.Lacking_complete_plumbing_facilities,econ.Public_transportation
;

""


### View 2

In [29]:
%%bigquery
CREATE VIEW reports.Covid_Impact_on_Service_Occupations AS
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day,econ.Service_occupations
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state, econ.Service_occupations
;

""


### View 3

In [ ]:
%%bigquery
CREATE VIEW reports.overall AS
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day, econ.Agriculture_forestry,(demo.White/demo.Total_Population) as pct_white
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state, econ.Agriculture_forestry,pct_white
;

### Overall Veiw
Combined for easier plotiing on data studio

In [23]:
%%bigquery
CREATE VIEW reports.overall AS
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day,house.Median_rooms, house.Lacking_complete_plumbing_facilities,econ.Unemployment_Rate,(econ.No_health_insurance_coverage/demo.Total_Population) as pct_with_no_health_insurance , social.Average_household_size, econ.Public_transportation, econ.Service_occupations, econ.Agriculture_forestry, econ.Median_earnings_for_workers, (demo.White/demo.Total_Population) as pct_white, (demo.Black/demo.Total_Population) as pct_black, (demo.Asian/demo.Total_Population) as pct_asian, (demo.Hispanic/demo.Total_Population) as pct_hispanic
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state,house.Median_rooms, house.Lacking_complete_plumbing_facilities,econ.Unemployment_Rate,pct_with_no_health_insurance, social.Average_household_size, econ.Public_transportation, econ.Service_occupations, econ.Agriculture_forestry, econ.Median_earnings_for_workers, pct_white,pct_black, pct_asian, pct_hispanic
;

""


In [22]:
%%bigquery
SELECT L.state,avg(C.deaths) as avg_deaths_per_day,avg(C.confirmed) as avg_cases_per_day,house.Median_rooms, house.Lacking_complete_plumbing_facilities,econ.Unemployment_Rate,(econ.No_health_insurance_coverage/demo.Total_Population) as pct_with_no_health_insurance , social.Average_household_size, econ.Public_transportation, econ.Service_occupations, econ.Agriculture_forestry, econ.Median_earnings_for_workers, (demo.White/demo.Total_Population) as pct_white, (demo.Black/demo.Total_Population) as pct_black, (demo.Asian/demo.Total_Population) as pct_asian, (demo.Hispanic/demo.Total_Population) as pct_hispanic
FROM datamart.location_Dataflow as L INNER JOIN datamart.day_Dataflow as D on L.combined_key=D.combined_key INNER JOIN datamart.cases_Dataflow as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state INNER JOIN datamart.housing as house on house.state = L.state INNER JOIN datamart.social as social on social.state = L.state
group by L.state,house.Median_rooms, house.Lacking_complete_plumbing_facilities,econ.Unemployment_Rate,pct_with_no_health_insurance, social.Average_household_size, econ.Public_transportation, econ.Service_occupations, econ.Agriculture_forestry, econ.Median_earnings_for_workers, pct_white,pct_black, pct_asian, pct_hispanic
;

,state,avg_deaths_per_day,avg_cases_per_day,Median_rooms,Lacking_complete_plumbing_facilities,Unemployment_Rate,pct_with_no_health_insurance,Average_household_size,Public_transportation,Service_occupations,Agriculture_forestry,Median_earnings_for_workers,pct_white,pct_black,pct_asian,pct_hispanic
0,Indiana,2.319485,55.569118,5.800000000,10233,4.200000000,0.085921,2.520000000,29628,539453,41079,33756,0.827647,0.095526,0.025336,0.072124
1,Minnesota,0.610169,14.992213,6.000000000,7180,3.200000000,0.048339,2.480000000,100645,479143,61997,40977,0.820548,0.065659,0.050647,0.055716
2,Oklahoma,0.931328,19.321805,5.500000000,6464,4.400000000,0.139712,2.570000000,7609,313570,82289,32323,0.724237,0.072635,0.022732,0.110718
3,Virginia,0.731587,26.850882,6.000000000,11024,4.000000000,0.077123,2.600000000,186834,723232,34735,40405,0.669514,0.194471,0.066269,0.097024
4,Vermont,1.427252,33.295612,5.500000000,1500,3.400000000,0.044458,2.280000000,5324,56435,8209,35474,0.937704,0.015120,0.013787,0.019885
5,Maryland,5.960383,211.010929,6.200000000,6054,4.500000000,0.059022,2.650000000,246926,532247,17758,45891,0.545333,0.302705,0.063879,0.106385
6,Tennessee,0.694259,39.449454,5.600000000,7098,4.500000000,0.099814,2.510000000,19604,546586,28342,32604,0.772115,0.167122,0.018412,0.056980
7,Louisiana,8.854914,200.845034,5.400000000,7479,5.500000000,0.086959,2.600000000,23798,404182,77379,32102,0.618394,0.323528,0.017533,0.053821
8,South Carolina,1.174800,47.588492,5.700000000,6473,4.700000000,0.106369,2.540000000,13837,414935,21668,32373,0.667053,0.264710,0.016814,0.057971
9,Alabama,0.922691,33.578313,5.700000000,6818,4.900000000,0.095632,2.520000000,8867,354923,26729,31962,0.678411,0.269121,0.013487,0.044725
